# Summary

This will cover calculating a baseline, calculating correction factors, 
and downscaling. The notebook assumed that you have run the prior notebooks
and have a tile directory with the following files in it:

  - `cru_AnnualTimeSeries/cru_AnnualTimeSeries-YYYY.nc`
  - `manifest.yml`
  - `worldclim.nc`

Start by loading libraries. 

In [1]:
# For development...
# %load_ext autoreload
# %autoreload 2

from temds import tile
from temds.datasources import crujra
from temds.datasources import worldclim

# Load the tile

This function will make an in memory `Tile` object with data from the directory passed to it.

In [2]:
mytile = tile.Tile.tile_from_directory("working/03-tiles/H00_V08")
  

In [3]:
for k,v in mytile.data.items():
  print(f"The tile has the key '{k}' to a variable of type {type(v)}")


The tile has the key 'worldclim' to a variable of type <class 'xarray.core.dataset.Dataset'>
The tile has the key 'cru_AnnualTimeSeries' to a variable of type <class 'temds.datasources.crujra.AnnualTimeSeries'>


In [4]:
#start_year = mytile.data['cru_AnnualTimeSeries'].range()[0]
#end_year = mytile.data['cru_AnnualTimeSeries'].range()[-1]

start_year = 1970
end_year = start_year + 30
print(f"Start year: {start_year}, End year: {end_year}")

Start year: 1970, End year: 2000


# Calculate the baseline

The docstring for `crujra.AnnualTimeSeries.create_climate_baseline()` is helpful:

In [5]:
print(crujra.AnnualTimeSeries.create_climate_baseline.__doc__)

Create baseline climate variables for dataset; uses
        the methods defined in CRUJRA_BASELINE_LOOKUP Based on original 
        downscaling.sh line 77-80. Here calculations are split up by var
        and the result is combined into a single dataset at the end.

        Algorithm: (pixel wise)
            (A) For each variable, daily data for each year in [start_year, 
        end_year] is averaged. 
            (B) For each month, the mean (or sum) of the daily average(from A)
        is calculated, giving the monthly baseline.
            (C) Monthly results are combined as time steps in yearly 
        dataset(xr.concat)
            (D) Each variables yearly dataset is combined into a single 
        dataset(xr.merge). This dataset is geo-referenced with crs from 
        first year of self.data.

        Parameters
        ----------
        start_year: int
            Inclusive start year for baseline
        end_year: int
            Inclusive end year for baseline

        

Once the baseline is computed, it is now in memory as part of the `Tile` object. If you save the object you will additionally get a baseline.nc file and that should be added to the manifest.

In [6]:
mytile.calculate_climate_baseline(start_year, start_year + 30, 'cru_baseline', 'cru_AnnualTimeSeries')
mytile.data['cru_baseline']

<xarray.Dataset> Size: 8MB
Dimensions:      (time: 12, x: 128, y: 140)
Coordinates:
  * time         (time) int64 96B 1 32 60 91 121 152 182 213 244 274 305 335
  * x            (x) float64 1kB -4.631e+06 -4.627e+06 ... -4.127e+06 -4.123e+06
  * y            (y) float64 1kB 1.93e+05 1.89e+05 ... -3.59e+05 -3.63e+05
    spatial_ref  int64 8B 0
Data variables:
    tmin         (time, y, x) float32 860kB 257.0 257.0 256.9 ... 253.5 253.4
    tmax         (time, y, x) float32 860kB 263.0 262.9 262.9 ... 259.6 259.5
    tmp          (time, y, x) float32 860kB 259.0 258.9 258.9 ... 255.7 255.7
    pre          (time, y, x) float32 860kB 262.7 262.6 262.5 ... 255.7 255.7
    dswrf        (time, y, x) float32 860kB 262.7 262.6 262.5 ... 255.7 255.7
    ugrd         (time, y, x) float32 860kB 1.133 1.136 1.14 ... 1.173 1.172
    vgrd         (time, y, x) float32 860kB -0.3147 -0.305 ... -0.02819 -0.02344
    spfh         (time, y, x) float32 860kB 0.000964 0.0009585 ... 0.0007636
    pres         (time, y, x) float32 860kB 9.637e+04 9.637e+04 ... 9.962e+04

# Correction Factors

Now that we have a climate baseline calcuated, we need to calculate correction
factors. The correction factors are handled slightly differently for different types
of variables (i.e. mean for some values, sum for others). Additionally the 
variables used are named slightly differently in the baseline and reference datasets.

So we start here by making a dictionary holding this info that we can pass to 
the correction factor function.

In [7]:
variables_cf = {
    'tmax': {'function': 'temperature', 'reference': 'tmax','baseline':'tmax', 'name': 'tmax'},
    'tmin': {'function': 'temperature', 'reference': 'tmin','baseline':'tmin', 'name': 'tmin'},
    'tavg': {'function': 'temperature', 'reference': 'tavg','baseline':'tmp', 'name': 'tavg'},
    'prec': {'function': 'precipitation', 'reference': 'prec','baseline':'pre', 'name': 'prec'},

    'vapo_pa': {
        'function': 'vapor-pressure', 
        'reference': 'vapr',
        'baseline-pres':'pres', 
        'baseline-spfh': 'spfh',
        'name': 'vapo'
        },
    'nirr': {
        'function': 'radiation', 
        'reference': 'srad',
        'baseline':'dswrf', 
        'name': 'nirr'
        
    },
    'ws': {
        'function': 'wind-speed', 
        'reference': 'wind',
        'baseline-ugrd': 'ugrd', 
        'baseline-vgrd': 'vgrd', 
        'name': 'ws'
        
    }
}

# Calculate correction factors for the variables
mytile.calculate_correction_factors('cru_baseline', 'worldclim', variables=variables_cf)

Now our tile object has a new `correction_factors` key in the data dictionary:

In [8]:
print(mytile.data.keys())
print()
print("Correction Factors:")
print(mytile.data['correction_factors'])

dict_keys(['worldclim', 'cru_AnnualTimeSeries', 'cru_baseline', 'correction_factors'])

Correction Factors:
<xarray.Dataset> Size: 6MB
Dimensions:      (y: 140, x: 128, time: 12)
Coordinates:
  * y            (y) float64 1kB 1.93e+05 1.89e+05 ... -3.59e+05 -3.63e+05
  * x            (x) float64 1kB -4.631e+06 -4.627e+06 ... -4.127e+06 -4.123e+06
  * time         (time) int64 96B 1 32 60 91 121 152 182 213 244 274 305 335
    spatial_ref  int64 8B 0
Data variables:
    tmax         (time, y, x) float32 860kB 0.7919 0.6752 0.5537 ... 1.202 1.231
    tmin         (time, y, x) float32 860kB -5.777 -5.874 ... -2.847 -2.75
    tavg         (time, y, x) float32 860kB -1.514 -1.59 ... -0.02271 0.03273
    prec         (time, y, x) float32 860kB 0.0836 0.08454 ... 0.1511 0.1489
    vapo         (time, y, x) float32 860kB 1.205 1.211 1.208 ... 1.39 1.391
    nirr         (time, y, x) float32 860kB 17.42 17.35 17.5 ... 10.3 10.24
    ws           (time, y, x) float32 860kB 3.022 2.983 2.962 ... 2

# Downscaling

The downscaling resamples the data to a finer resolution and applies the 
correction factors.

In [9]:
variables_ds = {
    'tmax': {'function': 'temperature', 'temperature': 'tmax','correction_factor':'tmax', 'name': 'tmax'},
    'tmin': {'function': 'temperature', 'temperature': 'tmin','correction_factor':'tmin', 'name': 'tmin'},
    'tavg': {'function': 'temperature', 'temperature': 'tmp','correction_factor':'tavg', 'name': 'tavg'},
    'prec': {'function': 'precipitation', 'precipitation': 'pre','correction_factor':'prec', 'name': 'prec'},
    'vapo': {'function': 'vapor-pressure', 'name': 'vapo', 'pres':'pres', 'spfh':'spfh', 'correction_factor': 'vapo' },
    'nirr': {'function':'radiation', 'name': 'nirr', 'dswrf':'dswrf','correction_factor': 'nirr'},
    'wind': {'function':'wind-speed', 'name': 'wind', 'ugrd': 'ugrd', 'vgrd':'vgrd', 'correction_factor':'ws'},
    'winddir': {'function':'wind-direction', 'name': 'winddir', 'ugrd': 'ugrd', 'vgrd':'vgrd' },
}

mytile.downscale_timeseries('downscaled_cru', 'cru_AnnualTimeSeries','correction_factors', variables_ds, True)




Once we have the downscaling done, we can save the data for the tile.



In [ ]:
# Make sure the output directory exists
!mkdir -p working/04-downscaled-tiles/
mytile.save("working/04-downscaled-tiles", items=['downscaled_cru', 'cru_baseline', 'correction_factors'], complevel=1)

python(82338) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [15]:
!ls working/04-downscaled-tiles/H00_V08/

python(88662) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


correction_factors.nc downscaled_cru
cru_baseline.nc       manifest.yml


# V ---- SCRATCH ---- V

In [11]:
type(mytile.crs)

import pyproj

pyproj.CRS.from_user_input(mytile.crs).to_wkt()

'PROJCRS["WGS 84 / NSIDC EASE-Grid 2.0 North",BASEGEOGCRS["WGS 84",ENSEMBLE["World Geodetic System 1984 ensemble",MEMBER["World Geodetic System 1984 (Transit)"],MEMBER["World Geodetic System 1984 (G730)"],MEMBER["World Geodetic System 1984 (G873)"],MEMBER["World Geodetic System 1984 (G1150)"],MEMBER["World Geodetic System 1984 (G1674)"],MEMBER["World Geodetic System 1984 (G1762)"],MEMBER["World Geodetic System 1984 (G2139)"],MEMBER["World Geodetic System 1984 (G2296)"],ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ENSEMBLEACCURACY[2.0]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["US NSIDC EASE-Grid 2.0 North",METHOD["Lambert Azimuthal Equal Area",ID["EPSG",9820]],PARAMETER["Latitude of natural origin",90,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],P